In [23]:
import pandas as pd
import numpy as np
import datetime
import gc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import median_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [24]:
df_train = pd.read_csv('../input/train.csv')
df_train = df_train.sort_values(by=['GameId','PlayId'])
print(df_train.shape)
display(df_train)

c:\users\0000011306852\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(509762, 49)


,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509757,2018123015,20181230154157,home,86.77,24.20,2.14,2.12,0.22,44.55,15.31,2555289,Nick Vannett,81,2018,23,4,00:16:00,SEA,2,8,ARZ,24,24,2558865,SINGLEBACK,"6 OL, 1 RB, 2 TE, 1 WR",9.0,"4 DL, 3 LB, 4 DB",right,2018-12-31T00:24:51.000Z,2018-12-31T00:24:49.000Z,11,6-6,261,03/06/1993,Ohio State,TE,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE
509758,2018123015,20181230154157,home,86.76,27.18,1.16,0.66,0.11,53.63,42.80,2555395,Germain Ifedi,65,2018,23,4,00:16:00,SEA,2,8,ARZ,24,24,2558865,SINGLEBACK,"6 OL, 1 RB, 2 TE, 1 WR",9.0,"4 DL, 3 LB, 4 DB",right,2018-12-31T00:24:51.000Z,2018-12-31T00:24:49.000Z,11,6-5,325,06/02/1994,Texas A&M,G,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE
509759,2018123015,20181230154157,home,87.26,27.05,2.59,1.18,0.26,3.96,21.12,2556524,George Fant,74,2018,23,4,00:16:00,SEA,2,8,ARZ,24,24,2558865,SINGLEBACK,"6 OL, 1 RB, 2 TE, 1 WR",9.0,"4 DL, 3 LB, 4 DB",right,2018-12-31T00:24:51.000Z,2018-12-31T00:24:49.000Z,11,6-5,322,07/19/1992,Western Kentucky,OT,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE
509760,2018123015,20181230154157,home,84.57,24.37,4.36,1.79,0.47,148.08,183.34,2557891,Ethan Pocic,77,2018,23,4,00:16:00,SEA,2,8,ARZ,24,24,2558865,SINGLEBACK,"6 OL, 1 RB, 2 TE, 1 WR",9.0,"4 DL, 3 LB, 4 DB",right,2018-12-31T00:24:51.000Z,2018-12-31T00:24:49.000Z,11,6-6,320,08/05/1995,Louisiana State,C,SEA,ARI,17

In [25]:
y_train = df_train[['PlayId','Yards']]
y_train = y_train.iloc[[i for i in range(y_train.shape[0]) if i%22==0]]
assert y_train.shape[0]==df_train['PlayId'].nunique()

df_train = df_train[[col for col in df_train.columns if col not in ['Yards']]]

### Some Correction
- https://www.kaggle.com/bgmello/neural-networks-feature-engineering-for-the-win

In [26]:
import re
from string import punctuation

def clean_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    txt = txt.replace('outside', 'outdoor')
    txt = txt.replace('outdor', 'outdoor')
    txt = txt.replace('outddors', 'outdoor')
    txt = txt.replace('outdoors', 'outdoor')
    txt = txt.replace('oudoor', 'outdoor')
    txt = txt.replace('indoors', 'indoor')
    txt = txt.replace('ourdoor', 'outdoor')
    txt = txt.replace('retractable', 'rtr.')
    return txt

def transform_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    if 'outdoor' in txt or 'open' in txt:
        return 1
    if 'indoor' in txt or 'closed' in txt:
        return 0
    
    return np.nan

In [27]:
#from https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/112681#latest-649087
Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
        'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
        'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
        'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 
        'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 

In [28]:
map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
for abb in df_train['PossessionTeam'].unique():
    map_abbr[abb] = abb

In [29]:
def height2cm(h):
    return int(h[0]) * 30.48 + int(h[2]) * 2.58

In [30]:
def str_to_float(txt):
    try:
        return float(txt)
    except:
        return -1

In [31]:
def clean_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = txt.replace('from', '')
    txt = txt.replace(' ', '')
    txt = txt.replace('north', 'n')
    txt = txt.replace('south', 's')
    txt = txt.replace('west', 'w')
    txt = txt.replace('east', 'e')
    return txt

def transform_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    
    if txt=='n':
        return 0
    if txt=='nne' or txt=='nen':
        return 1/8
    if txt=='ne':
        return 2/8
    if txt=='ene' or txt=='nee':
        return 3/8
    if txt=='e':
        return 4/8
    if txt=='ese' or txt=='see':
        return 5/8
    if txt=='se':
        return 6/8
    if txt=='ses' or txt=='sse':
        return 7/8
    if txt=='s':
        return 8/8
    if txt=='ssw' or txt=='sws':
        return 9/8
    if txt=='sw':
        return 10/8
    if txt=='sww' or txt=='wsw':
        return 11/8
    if txt=='w':
        return 12/8
    if txt=='wnw' or txt=='nww':
        return 13/8
    if txt=='nw':
        return 14/8
    if txt=='nwn' or txt=='nnw':
        return 15/8
    return np.nan

In [32]:
def map_weather(txt):
    """
    climate controlled or indoor => 3, sunny or sun => 2, clear => 1, cloudy => -1, rain => -2, snow => -3, others => 0
    partly => multiply by 0.5
    """
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

In [33]:
def correction_df(df):
    df['StadiumType'] = df['StadiumType'].apply(clean_StadiumType)
    df['StadiumType'] = df['StadiumType'].apply(transform_StadiumType)
    
    df['Turf'] = df['Turf'].map(Turf)
    df['Turf'] = (df['Turf'] == 'Natural') * 1
    
    df['PossessionTeam'] = df['PossessionTeam'].map(map_abbr)
    df['HomeTeamAbbr'] = df['HomeTeamAbbr'].map(map_abbr)
    df['VisitorTeamAbbr'] = df['VisitorTeamAbbr'].map(map_abbr)

    df['HomePossesion'] = (df['PossessionTeam'] == df['HomeTeamAbbr']) * 1
    df['Field_eq_Possession'] = (df['FieldPosition'] == df['PossessionTeam']) * 1
    df['HomeField'] = (df['FieldPosition'] == df['HomeTeamAbbr']) * 1
    
    t_gemeclock = pd.to_datetime(df['GameClock'])
    df['GameClock'] = t_gemeclock.dt.minute*60 + t_gemeclock.dt.second
    
    df['PlayerHeight'] = df['PlayerHeight'].map(height2cm)
    
    df['PlayerBirthDate'] = pd.to_datetime(df['PlayerBirthDate'])
    df['Age'] = 2019 - df['PlayerBirthDate'].dt.year
    
    t_handoff = pd.to_datetime(df['TimeHandoff'])
    t_handsnap = pd.to_datetime(df['TimeSnap'])
    df['TimeDelta'] = (t_handoff.dt.minute*60 + t_handoff.dt.second) - (t_handsnap.dt.minute*60 + t_handsnap.dt.second)
    
    # remove mph
    # replace the ones that has x-y by (x+y)/2
    # and also the ones with x gusts up to y
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: str(x).lower().replace('mph', '').strip() if not pd.isna(x) else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(str_to_float)
    
    df['WindDirection'] = df['WindDirection'].apply(clean_WindDirection)
    df['WindDirection'] = df['WindDirection'].apply(transform_WindDirection)
    
    # binary variables to 0/1
    df['PlayDirection'] = (df['PlayDirection'].apply(lambda x: x.strip() == 'right')) * 1
    df['Team'] = (df['Team'].apply(lambda x: x.strip()=='home')) * 1
    
    df['GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(map_weather)
    
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher']) * 1
    
    df['YardsLeft'] = df.apply(lambda row: 100-row['YardLine'] if row['HomeField'] else row['YardLine'], axis=1)
    df['YardsLeft'] = df.apply(lambda row: row['YardsLeft'] if row['PlayDirection'] else 100-row['YardsLeft'], axis=1)
    
    return df

In [34]:
def make_features(df):
    df['DefendersInTheBox_vs_Distance'] = df['DefendersInTheBox'] / df['Distance']
    
    X_list = []
    for val in df[df['NflId']==df['NflIdRusher']]['X']:  # X of NflIdRusher in each PlayId
        X_temp_list = [val]*22
        X_list.extend(X_temp_list)

    Y_list = []
    for val in df[df['NflId']==df['NflIdRusher']]['Y']:  # Y of NflIdRusher in each PlayId
        Y_temp_list = [val]*22
        Y_list.extend(Y_temp_list)

    df['X_Rusher'] = X_list
    df['Y_Rusher'] = Y_list

    df['DisFromRusher'] = np.sqrt((df['X_Rusher']-df['X'])**2 + (df['Y_Rusher']-df['Y'])**2)
    
    return df
    
def make_df_play_uq(df):
    # unique columns in each PlayId
    cols_play_uq = ['Season','YardLine','Quarter','GameClock','Down','Distance','HomeScoreBeforePlay','VisitorScoreBeforePlay','DefendersInTheBox','PlayDirection','Week','StadiumType','Turf','GameWeather','Temperature','Humidity','WindSpeed','WindDirection','HomePossesion','Field_eq_Possession','HomeField','TimeDelta','YardsLeft']
    add_cols = ['DefendersInTheBox_vs_Distance']
    
    cols_play_uq.extend(add_cols) 
    cols_play_uq_cat = ['OffenseFormation','OffensePersonnel','DefensePersonnel']
    for col in cols_play_uq_cat:
        for col_dum in df.columns[df.columns.str.contains('%s_'%col)]:
            cols_play_uq.append(col_dum)

    for col in cols_play_uq:
        assert df[['PlayId',col]].groupby('PlayId')[col].nunique(dropna=False).sort_values(ascending=False).sum()==df['PlayId'].nunique()

    df_play_uq = df[['PlayId']+cols_play_uq].iloc[[i for i in range(df.shape[0]) if i%22==0]]
    assert df['PlayId'].nunique()==df_play_uq.shape[0]
    
    return df_play_uq

In [35]:
def features_agg(df, id_col, agg_func):
    """
    注意として、入力のdfに対して、出力は集計されたid_colに対して集計されたdataframeを返す
    agg_func: {col名: ['mean', 'max', 'min', 'std',...], }のdict
    """
    df_agg = df.groupby(id_col).agg(agg_func)
    df_agg.columns = ['_'.join(col) for col in df_agg.columns.values]
    df_agg.reset_index(inplace=True)
    df_agg.fillna(0, inplace=True)

    return df_agg

# cols_playerに関するカラムの集計
agg_player = {
    'X': ['mean','median','max','min','std'],
    'Y': ['mean','median','max','min','std'],
    'S': ['mean','median','max','min','std'],
    'A': ['mean','median','max','min','std'],
    'Dis': ['mean','median','max','min','std'],
    'Orientation': ['mean','median','max','min','std'],
    'Dir': ['mean','median','max','min','std'],
    'PlayerHeight': ['mean','median','max','min','std'],
    'PlayerWeight': ['mean','median','max','min','std'],
    'Age': ['mean','median','max','min','std'],
    'DisFromRusher': ['mean','median','max','min','std'],
}

In [36]:
def transform_df(df):
    df = df.sort_values(by=['GameId','PlayId'])
    
    df = correction_df(df)
    df = make_features(df)
    
    # one-hot encoding
    cols_cat = ['FieldPosition','JerseyNumber','OffenseFormation','OffensePersonnel','DefensePersonnel','Position']
    df = pd.get_dummies(df, columns=cols_cat, drop_first=True, dummy_na=True)

    df_play_uq = make_df_play_uq(df)
    
    cols_player = ['X','Y','S','A','Dis','Orientation','Dir','PlayerHeight','PlayerWeight','Age','DisFromRusher']
    cols_player_cat = ['JerseyNumber','Position']
    for col in cols_player_cat:
        for col_dum in df.columns[df.columns.str.contains('%s_'%col)]:
            cols_player.append(col_dum)
    df_each_player = pd.DataFrame(np.array(df[cols_player]).reshape(-1,len(cols_player)*22), columns=[col+'_away_%d'%i for col in cols_player for i in range(1,12)]+[col+'_home_%d'%i for col in cols_player for i in range(1,12)])
    df_each_player['PlayId'] = df['PlayId'].unique()
    
    # add one-hot columns
    cols_player_cat = ['JerseyNumber','Position']
    for col in cols_player_cat:
        for col_dum in df.columns[df.columns.str.contains('%s_'%col)]:
            agg_player[col_dum] = ['sum']
    df_agg_player_team = features_agg(df, ['PlayId','Team'], agg_player)
    
    usecols = [col for col in df_agg_player_team.columns if col not in ['PlayId','Team']]
    df_agg_player = pd.DataFrame(np.array(df_agg_player_team[usecols]).reshape(-1,len(usecols)*2), columns=[col+'_away' for col in usecols]+[col+'_home' for col in usecols])
    df_agg_player['PlayId'] = df_agg_player_team['PlayId'].unique()
    df_agg_player
    
    # 結合
    df = pd.merge(df_play_uq, df_each_player, on='PlayId', validate='1:1')
    df = pd.merge(df, df_agg_player, on='PlayId', validate='1:1')
    
    del df_play_uq, df_each_player, df_agg_player_team, df_agg_player
    gc.collect()
    
    return df

In [37]:
df_train = transform_df(df_train)
df_train

PlayId  Season  YardLine  Quarter  GameClock  Down  Distance  \
0      20170907000118    2017        35        1        840     3         2   
1      20170907000139    2017        43        1       3120     1        10   
2      20170907000189    2017        35        1        120     1        10   
3      20170907000345    2017         2        1        720     2         2   
4      20170907000395    2017        25        1        480     1        10   
...               ...     ...       ...      ...        ...   ...       ...   
23166  20181230153910    2018        35        4        180     1        10   
23167  20181230154035    2018        25        4       2940     1        10   
23168  20181230154082    2018        34        4       1440     3         1   
23169  20181230154135    2018        25        4       3360     1        10   
23170  20181230154157    2018        23        4        960     2         8   

       HomeScoreBeforePlay  VisitorScoreBeforePlay  DefendersInTheBox  \
0                        0                       0                6.0   
1                        0                       0                6.0   
2                        0                       0                7.0   
3                        0                       0                9.0   
4                        7                       0                7.0   
...                    ...                     ...                ...   
23166                   24                      21                8.0   
23167                   24                      24                6.0   
23168                   24                      24                7.0   
23169                   24                      24                7.0   
23170                   24                      24                9.0   

       PlayDirection  Week  StadiumType  Turf  GameWeather  Temperature  \
0                  0     1          1.0     0          1.0         63.0   
1                  0     1          1.0     0          1.0         63.0   
2                  0     1          1.0     0          1.0         63.0   
3                  0     1          1.0     0          1.0         63.0   
4                  1     1          1.0     0          1.0         63.0   
...              ...   ...          ...   ...          ...          ...   
23166              0    17          1.0     0         -1.0         45.0   
23167              1    17          1.0     0         -1.0         45.0   
23168              1    17          1.0     0         -1.0         45.0   
23169              1    17          1.0     0         -1.0         45.0   
23170              1    17          1.0     0         -1.0         45.0   

       Humidity  WindSpeed  WindDirection  HomePossesion  Field_eq_Possession  \
0          77.0        8.0           1.25              1                    1   
1          77.0        8.0           1.25              1                    1   
2          77.0        8.0           1.25              1                    0   
3          77.0        8.0           1.25              1                    0   
4          77.0        8.0           1.25              0                    1   
...         ...        ...            ...            ...                  ...   
23166      76.0        5.0           0.75              0                    0   
23167      76.0        5.0           0.75              1                    1   
23168      76.0        5.0           0.75              1                    1   
23169      76.0        5.0           0.75              1                    0   
23170      76.0        5.0           0.75              1                    0   

       HomeField  TimeDelta  YardsLeft  DefendersInTheBox_vs_Distance  \
0              1          1         35                          3.000   
1              1          1         43                          0.600   
2              0          2         65                          0.700   
3              0       

In [38]:
cols_rm = ['PlayId','Yards']
features = [col for col in df_train.columns if col not in cols_rm]


df_train = pd.merge(df_train, y_train, on='PlayId', validate='1:1')
X_train = df_train[features]
y_train = df_train['Yards'] + 99

In [39]:
params = {
    'objective':'multiclass',
    'num_class': 199,
    "boosting": "gbdt",
    #"metric": 'mae',
    'learning_rate': 0.01,
    'num_leaves': 63,
    'max_depth': -1,
    # 'min_data_in_leaf': 30, 
    # "min_child_samples": 20,
    # "feature_fraction": 0.9,
    # "bagging_freq": 1,
    # "bagging_fraction": 0.9 ,
    # "bagging_seed": 11,
    # "lambda_l1": 0.1,
    "verbosity": -1,
    "nthread": -1,
    "random_state": 16
}

In [41]:
from sklearn.model_selection import RepeatedKFold, KFold, StratifiedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=16)

models = []
oof = np.zeros((509762//22, 199))
feature_importance_df = pd.DataFrame()
for fold_, (tr_idx, val_idx) in enumerate(folds.split(X_train.values, y_train.values)):
    print('-')
    print("Fold {}".format(fold_ + 1))
    
    tr_data = lgb.Dataset(X_train.iloc[tr_idx], label=y_train[tr_idx])
    val_data = lgb.Dataset(X_train.iloc[val_idx], label=y_train[val_idx])

    num_round = 10
    clf = lgb.train(params, tr_data, num_round, valid_sets = [tr_data, val_data], verbose_eval=100, early_stopping_rounds=100)
    oof[val_idx, :] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
    
    models.append(clf)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

c:\users\0000011306852\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


-
Fold 1
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's multi_logloss: 2.67358	valid_1's multi_logloss: 2.90988
-
Fold 2
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's multi_logloss: 2.674	valid_1's multi_logloss: 2.90835
-
Fold 3
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's multi_logloss: 2.67693	valid_1's multi_logloss: 2.89428
-
Fold 4
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's multi_logloss: 2.67871	valid_1's multi_logloss: 2.86729
-
Fold 5
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's multi_logloss: 2.67886	valid_1's multi_logloss: 2.86854


In [49]:
oof

array([[9.73375410e-16, 9.73375410e-16, 9.73375410e-16, ...,
        5.00243602e-05, 9.73375410e-16, 5.00243603e-05],
       [9.97882231e-16, 9.97882231e-16, 9.97882231e-16, ...,
        5.11594500e-05, 9.97882231e-16, 5.11594500e-05],
       [1.00821429e-15, 1.00821429e-15, 1.00821429e-15, ...,
        5.18148233e-05, 1.00821429e-15, 5.18148233e-05],
       ...,
       [1.00176147e-15, 1.00176147e-15, 1.00176147e-15, ...,
        5.13583310e-05, 1.00176147e-15, 5.13583307e-05],
       [1.00702185e-15, 1.00702185e-15, 1.00702185e-15, ...,
        5.17535407e-05, 1.00702185e-15, 5.17535407e-05],
       [1.00406905e-15, 1.00406905e-15, 1.00406905e-15, ...,
        1.00406905e-15, 1.00406905e-15, 5.15294038e-05]])

In [48]:
y_ans = np.zeros((509762//22, 199))
for i,p in enumerate(y_train-99):
    p+=99
    for j in range(199):
        if j>=p:
            y_ans[i][j]=1.0

print("validation score:",np.sum(np.power(np.cumsum(oof, axis=1)-y_ans, 2))/(199*(509762//22)))

validation score: 0.014071518381040433


In [ ]:
# output importance
feature_importance_mean = feature_importance_df[['feature','importance']].groupby('feature').mean().sort_values(by="importance", ascending=False)
print(feature_importance_mean.head(20))
feature_importance_mean.to_csv("../importance/lgbm.csv", index=True)

In [ ]:
feature_importance_mean[feature_importance_mean['importance']==0].index

In [ ]:
with open("../importance/null_features.txt", 'wt') as f:
    for ele in list(feature_importance_mean[feature_importance_mean['importance']==0].index):
      f.write('"'+ele+'",'+'\n')

In [ ]:
y=np.array([[0.5,0.6,0.7,0.55,0.45,0.77],[0.5,0.2,0.3,0.55,0.45,0.77],[0.5,0.6,0.7,0.45,0.35,0.77]])
y = np.mean(y, axis=0)
print(y)
y = np.cumsum(y, axis=0).clip(0,1)
y

In [ ]:
ab = np.cumsum(0.4, axis=0)#.clip(0, 1)
aa+=ab
aa

In [ ]:
def make_pred(df, sample, env, models):
    df = transform_df(df)
    
    y_pred = np.mean([model.predict(df) for model in models], axis=0)
    y_pred = np.cumsum(y_pred, axis=0).clip(0, 1)
    for pred in y_pred:
        prev=0
        for i in range(len(pred)):
            if prev>pred[i]:
                pred[i]=prev
            prev=pred[i]
    yardsleft = np.array(df['YardsLeft'][::22])
    
    for i in range(len(yardsleft)):
        y_pred[i, :yardsleft[i]-1] = np.zeros(shape=y_pred[i, :yardsleft[i]-1].shape)
        y_pred[i, yardsleft[i]+100:] = np.ones(shape=y_pred[i, yardsleft[i]+100:].shape)
    env.predict(pd.DataFrame(data=y_pred, columns=sample.columns))
    return y_pred

In [ ]:
for (test_df, sample_prediction_df) in env.iter_test():
    make_pred(test, sample, env, model)

In [ ]:
env.write_submission_file()